<a href="https://colab.research.google.com/github/semishen/ML100Days/blob/master/Day_012_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察填補缺值以及 標準化 / 最小最大化 對數值的影響

# [作業重點]
- 觀察替換不同補缺方式, 對於特徵的影響 (In[4]~In[6], Out[4]~Out[6])
- 觀察替換不同特徵縮放方式, 對於特徵的影響 (In[7]~In[8], Out[7]~Out[8])

In [26]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

df_train = pd.read_csv('titanic_train.csv')
df_test = pd.read_csv('titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
print('shape of df_train: ', df_train.shape)
print('shape of df_test: ',df_test.shape)
print('shape of df: ',df.shape)
df.head()

shape of df_train:  (891, 10)
shape of df_test:  (418, 10)
shape of df:  (1309, 10)


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [27]:
df.dtypes.value_counts()

object     5
int64      3
float64    2
dtype: int64

In [28]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中

num_features = list(df.columns.values[(df.dtypes == 'float64') | (df.dtypes == 'int64')])
num_features
# num_features = []
# for dtype, feature in zip(df.dtypes, df.columns):
#     if dtype == 'float64' or dtype == 'int64':
#         num_features.append(feature)
# print(f'{len(num_features)} Numeric Features : {num_features}\n')

['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [29]:
# 削減文字型欄位, 只剩數值型欄位
df_num = df[num_features]
train_num = train_Y.shape[0]
df_num.head()

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


In [36]:
df_num.isna().sum()

Pclass      0
Age       263
SibSp       0
Parch       0
Fare        1
dtype: int64

# 作業1
* 試著在補空值區塊, 替換並執行兩種以上填補的缺值, 看看何者比較好?

In [31]:
# 空值補 -1, 做羅吉斯迴歸
df_m1 = df_num.fillna(-1)
train_X = df_m1[:train_num]
estimator = LogisticRegression()
score_m1 = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

# 空值補 mean, 做羅吉斯迴歸
df_mean = df_num.fillna(df.mean())
train_X = df_mean[:train_num]
estimator = LogisticRegression()
score_mean = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

# 空值補 median, 做羅吉斯迴歸
df_median = df_num.fillna(df.median())
train_X = df_median[:train_num]
estimator = LogisticRegression()
score_median = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

print("score of fill in -1: ", score_m1)
print("score of fill in mean: ", score_mean)
print("score of fill in median: ", score_median)

score of fill in -1:  0.6981796497395015
score of fill in mean:  0.6947900320130562
score of fill in median:  0.6947837549431924


### 分別以 -1, mean, median 填補缺值，結果填補 -1 所得到的分數最高。

# 作業2
* 使用不同的標準化方式 ( 原值 / 最小最大化 / 標準化 )，搭配羅吉斯迴歸模型，何者效果最好?

In [45]:
# 空值補 -1, 原值 做羅吉斯迴歸
df_m1 = df_num.fillna(-1)
train_X = df_m1[:train_num]
estimator = LogisticRegression()
score_noscale = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
print("score of fill in -1, no scale: ", score_noscale)

# 空值補 -1, 最小最大化 做羅吉斯迴歸
df_m1 = df_num.fillna(-1)
df_minmax = MinMaxScaler().fit_transform(df_m1)
train_X = df_minmax[:train_num]
estimator = LogisticRegression()
score_minmax = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
print("score of fill in -1, minmax scaler: ", score_minmax)

# 空值補 -1, 標準化 做羅吉斯迴歸
df_m1 = df_num.fillna(-1)
df_std = StandardScaler().fit_transform(df_m1)
train_X = df_std[:train_num]
estimator = LogisticRegression()
score_std = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
print("score of fill in -1, std scaler: ", score_std)

score of fill in -1, no scale:  0.6981796497395015
score of fill in -1, minmax scaler:  0.6970560542338836
score of fill in -1, std scaler:  0.6981733726696377


### 分別以 原值 最小最大化 標準化 進行標準化，結果原值所得到的分數最高。